## Before submitting
1. Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

2. Make sure that no assertions fail or exceptions occur, otherwise points will be subtracted.

3. Please submit only the `*.ipynb` file.

4. Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE". Edit only between `YOUR CODE HERE` and `END YOUR CODE`.

5. Fill your group name and collaborators below:

In [63]:
GROUPNAME = "35"
COLLABORATORS = "Yevhen Yazvinskyi 396417, Sourabh Raj 397371, Yizhou Peng 398821"

---

# Sheet 3: Rounding, Overflow, Linear Algebra

In this exercise sheet, we look at various sources of numerical overflow when executing Python and numpy code for large input values, and how to efficiently handle them, for example, by using numpy special functions.

In [64]:
import numpy,utils
import numpy as np

## Building a robust "softplus" nonlinear function (30 P)

The softplus function is defined as:

$$
\mathrm{softplus}(x) = \log(1+\exp(x)).
$$

It intervenes as elementary computation in certain machine learning models such as neural networks. Plotting it gives the following curve

![plot generated using fooplot.com](softplus.png)

where the function tends to zero for very negative input values and tends to the identity for very positive input values.

In [65]:
def softplus(z): return numpy.log(1+numpy.exp(z))

We consider an input vector from the module `utils` containing varying values between 1 and 10000. We would like to apply the `softplus` function to all of its element in an element-wise manner.

In [66]:
X = utils.softplus_inputs
print(X)

[-10000, -1000, -100, -10, -1, 0, 1, 10, 100, 1000, 10000]


We choose these large values in order to test whether the behavior of the function is correct in all regimes of the function, in particular, for very small or very large values. The code below applies the `softplus` function directly to the vector of inputs and then prints for all cases the input and the corresponding function output:

In [67]:
Y = softplus(X)
for x,y in zip(X,Y):
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


/home/ydjeen/Homework/Python/pyml-virt/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


For large input values, the softplus function returns `inf` whereas analysis of that function tells us that it should compute the identity. Let's now try to apply the softplus function one element at a time, to see whether the problem comes from numpy arrays:

In [68]:
for x in X:
    y = softplus(x)
    print('softplus(%11.4f) = %11.4f'%(x,y))

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =         inf
softplus( 10000.0000) =         inf


/home/ydjeen/Homework/Python/pyml-virt/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


Unfortunately, the result is the same. We observe that the function always stops working when its output approaches 1000, even though the input was given in high precision `float64`.

* Create an alternative function for `softplus_robust` that applies to input scalars and that correctly applies to values that can be much larger than 1000 (e.g. billions or more). Your function can be written in Python directly and does not need numpy parallelization.

In [69]:
def softplus_robust(x):
    # >>>>> YOUR CODE HERE
    # another aproach
    #if (x > 100):
    #    return x
    #if (x < -10):
    #    return 0
    #return softplus(x)
    if x<0 :
      return numpy.log(1 + numpy.exp(x));
    else :
      return x + numpy.log(numpy.exp(-x) +1);
    # <<<<< END YOUR CODE

In [70]:
# Verify your function
for x in X:
    y = softplus_robust(x)
    print('softplus(%11.4f) = %11.4f'%(x,y))


softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000


As we have seen in the previous exercise sheet, the problem of functions that apply to scalars only is that they are less efficient than functions that apply to vectors directly. Therefore, we would like to handle the rounding issue directly at the vector level.

* Create a new softplus function that applies to vectors and that has the desired behavior for large input values. Your function should be fast for large input vectors (i.e. it is not appropriate to use an inner Python loop inside the function).

In [71]:
def softplus_robust_vec(X):
    # >>>>> YOUR CODE HERE
    softplus_vectorised = np.vectorize(softplus_robust)
    return softplus_vectorised(X)
    # <<<<< END YOUR CODE

In [72]:
# Verify your function
Y = softplus_robust_vec(X)
Tuples = tuple(zip(X,Y))
for tup in Tuples:
    print('softplus(%11.4f) = %11.4f'%tup)

softplus(-10000.0000) =      0.0000
softplus( -1000.0000) =      0.0000
softplus(  -100.0000) =      0.0000
softplus(   -10.0000) =      0.0000
softplus(    -1.0000) =      0.3133
softplus(     0.0000) =      0.6931
softplus(     1.0000) =      1.3133
softplus(    10.0000) =     10.0000
softplus(   100.0000) =    100.0000
softplus(  1000.0000) =   1000.0000
softplus( 10000.0000) =  10000.0000


## Computing a partition function (40 P)

We consider a discrete probability distribution of type
$$
p(\boldsymbol{x};\boldsymbol{w}) = \frac{1}{Z(\boldsymbol{w})} \exp(\boldsymbol{x}^\top \boldsymbol{w})
$$
where $\boldsymbol{x} \in \{-1,1\}^{10}$ is an observation, and $\boldsymbol{w} \in \mathbb{R}^{10}$ is a vector of parameters. The term $Z(\boldsymbol{w})$ is called the partition function and is chosen such that the probability distribution sums to 1. That is, the equation:
$$
\sum_{\boldsymbol{x} \in \{-1,1\}^{10}} p(\boldsymbol{x};\boldsymbol{w}) = 1
$$
must be satisfied. Below is a simple method that computes the log of the partition function $Z(\boldsymbol{w})$ for various choices of parameter vectors. The considered parameters (`w_small`, `w_medium`, and `w_large`) are increasingly large (and thus problematic), and can be found in the file `utils.py`.

In [73]:
import numpy,utils
import itertools

def getlogZ(w):
    Z = 0
    for x in itertools.product([-1, 1], repeat=10):
        Z += numpy.exp(numpy.dot(x,w))
    return numpy.log(Z)

print('%11.4f'%getlogZ(utils.w_small))
print('%11.4f'%getlogZ(utils.w_medium))
print('%11.4f'%getlogZ(utils.w_big))

    17.2457
    89.5932
        inf


/home/ydjeen/Homework/Python/pyml-virt/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


We can observe from these results, that for parameter vectors with large values (e.g. `utils.w_big`), the exponential function overflows, and thus, we do not obtain a correct value for the logarithm of `Z`.

* Implement an improved function  `getlogZ_robust` that avoids the overflow problem, and evaluates the partition function for the same parameters.

In [74]:
import itertools

def getlogZ_robust(w):
    # >>>>> YOUR CODE HERE
    # another aproach
    #Z = 0
    #X = np.arange(1024, dtype='float64')
    #i = 0
    #for x in itertools.product([-1, 1], repeat=10):
    #    X[i] = numpy.dot(x,w)
    #    i+=1
    #xmax = np.max(X)
    #for x in X:
    #    Z += numpy.exp(x - xmax)
    #return xmax + numpy.log(Z)
    z = 0
    for x in itertools.product([-1, 1], repeat=10):
        z = np.logaddexp(z, np.dot(x, w))
    return z;
    # <<<<< END YOUR CODE

In [75]:
# Verify your function
print('%11.4f'%getlogZ_robust(utils.w_small))
print('%11.4f'%getlogZ_robust(utils.w_medium))
print('%11.4f'%getlogZ_robust(utils.w_big))

    17.2457
    89.5932
 24919.9913


* For the model with parameter `utils.w_big`, evaluate the log-probability of the binary vectors contained in the list `itertools.product([-1, 1], repeat=10)`, and return a `numpy` array of the indices (starting from 0) of those that have probability greater or equal to 0.001.

In [76]:
def important_indexes(tol = 0.001):
    Z = getlogZ_robust(utils.w_big)
    # >>>>> YOUR CODE HERE
    P = list()
    count = 0;
    for x in itertools.product([-1, 1], repeat=10):
        p = numpy.exp(np.dot(x,utils.w_big) - Z)
        if p >= tol :
            P.append(count)
        count+=1;
    return P;
    # <<<<< END YOUR CODE

In [77]:
# Verify your function
print(important_indexes())

[81, 83, 85, 87, 209, 211, 213, 215, 337, 339, 341, 343, 465, 467, 469, 471, 597, 599, 725, 727, 853, 855, 981, 983]


## Probability of generating data from a Gaussian model (30 P)

Consider a multivariate Gaussian distribution of mean vector `m` and covariance `S`. The probability associated to a vector `x` is given by:

$$
p(\boldsymbol{x};(\boldsymbol{m},S)) = \frac{1}{\sqrt{(2\pi)^d \mathrm{det}(S)}} \exp \Big( - \frac12 (\boldsymbol{x}-\boldsymbol{m})^\top S^{-1} (\boldsymbol{x}-\boldsymbol{m})\Big)
$$

We consider the calculation of the probability of observing a certain dataset 

$$
\mathcal{D} = (\boldsymbol{x}^{(1)},\dots,\boldsymbol{x}^{(N)})
$$

assuming the data is generated according to a Gaussian distribution of fixed parameters $\boldsymbol{m}$ and $S$. Such probability density is given by the formula:

$$
\log P(\mathcal{D};(\boldsymbol{m},S)) = \log \prod_{i=1}^N p(\boldsymbol{x}^{(i)};(\boldsymbol{m},S))
$$

The function below implements such function:

In [78]:
import numpy,numpy.linalg,utils

def logp(X,m,S):
    
    # Find the number of dimensions from the data vector
    d = X.shape[1]
    
    # Invert the covariance matrix
    Sinv = numpy.linalg.inv(S)
    
    # Compute the quadratic terms for all data points
    Q = -0.5*(numpy.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    # Raise them quadratic terms to the exponential
    Q = numpy.exp(Q)
    
    # Divide by the terms in the denominator
    P = Q / numpy.sqrt((2*numpy.pi)**d * numpy.linalg.det(S))
    
    # Take the product of the probability of each data points
    Pprod = numpy.prod(P)
    
    # Return the log-probability
    return numpy.log(Pprod)


Evaluation of this function for various datasets and parameters provided in the file `utils.py` gives the following probabilities:

In [79]:
print('%11.4f'%logp(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp(utils.X3,utils.m3,utils.S3))

   -13.0068
       -inf
       -inf


/home/ydjeen/Homework/Python/pyml-virt/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in log


This function is numerically instable for multiple reasons. The product of many probabilities, the inversion of a large covariance matrix, and the computation of its determinant, are all potential causes for overflow. Thus, we would like to find a numerically robust way of performing each of these.

* Implement a numerically stable version of the function `logp`
* Evaluate it on the same datasets and parameters as the function `logp`

In [80]:
def logp_robust(X,m,S):
    # >>>>> YOUR CODE HERE
    
    d = X.shape[1]
    
    Sinv = numpy.linalg.inv(S)
    
    Q = -0.5*(numpy.dot(X-m,Sinv)*(X-m)).sum(axis=1)
    
    # ln( (denom*exp(X)).productAll() ) = (ln(denom*exp(X))).sumAll() = (ln(denom) + X).sumAll()
    
    denom = numpy.log(1/(numpy.sqrt((2*numpy.pi)**d * numpy.linalg.det(S))))
    
    return numpy.sum(Q+denom)
    # <<<<< END YOUR CODE

In [81]:
# Verify your function
print('%11.4f'%logp_robust(utils.X1,utils.m1,utils.S1))
print('%11.4f'%logp_robust(utils.X2,utils.m2,utils.S2))
print('%11.4f'%logp_robust(utils.X3,utils.m3,utils.S3))

   -13.0068
 -1947.9710
-218646.1739
